# Exponential Value Approximation (EVA)



The first step will be to generate the Hamiltonian with which we will test. Here is shown the construction of a Hamiltonian of order 3 in which each term with probability p will appear. In case we want to test with the Hamiltonian of order 2 we will have to comment the last for.

In [1]:
import pennylane as qml
from pennylane import numpy as np

p = 0.75 
size = 10 # number of qubits

obs = []
for j in range(size):
    if np.random.rand() < p :obs.append(qml.PauliZ(wires = j))
        
    for i in range(j+1,size):
        if np.random.rand() < p :obs.append(qml.PauliZ(wires = j) @ qml.PauliZ(wires = i))
            
        for k in range(i+1,size):
            if np.random.rand() < p :obs.append(qml.PauliZ(wires = j) @ qml.PauliZ(wires = i) @ qml.PauliZ(wires = k))
        

coefs = (np.random.rand(len(obs))-0.5)*2
coefs = coefs / sum([abs(i) for i in coefs]) # we normalize the coefficients


H = qml.Hamiltonian(coefs, obs)

Our objective will be to calculate the expected value of a state $\phi$ through the Hamiltonian. To do this we will first define the ansantz that we will use to generate the $\phi$ state.

In [2]:
w = np.random.rand(size) * np.pi

def ansantz(w, wires = list(range(size))):
    for i in wires:
        qml.RX(w[i], wires = i)
        qml.CNOT(wires = [i, (i + 1) % size])

We will start by implementing EVA. Recall that it had the following structure:


<img src="EVA.png">

In [3]:
from pennylane import numpy as np
from pennylane.templates import ApproxTimeEvolution

shots = 5000
k = 4
       
@qml.template
def evolution():
    ApproxTimeEvolution(H, -1/k, 1)
    
ops = qml.ctrl(evolution, control = size)
dev = qml.device("default.qubit", size + 1, shots = int(len(H.coeffs) * shots * k ** 2))

@qml.qnode(dev)
def model(w):
    ansantz(w)
    qml.Hadamard(wires = size)
    qml.RZ(-np.pi/2, wires = size)
    ops()
    qml.Hadamard(wires = size)
    return qml.expval(qml.PauliZ(wires = size))

def EVA(w):
    return model(w) * k 

print("expected value:", EVA(w))

expected value: -0.04998235294117647


We will now move on to implement the second version, the reduced EVA. In this case the circuit was as follows:

<img src="rEVA.png">

In [4]:
@qml.template
def hadamards():
    for i in range(size):
        qml.Hadamard(wires = i)
    
    
ops2 = qml.ctrl(hadamards, control = size)

@qml.qnode(dev)
def model2(w):
    ansantz(w)
    qml.Hadamard(wires = size)
    qml.RZ(np.pi/2, wires = size)
    ops2()
    ApproxTimeEvolution(H, -1/k, 1)
    ops2()
    qml.Hadamard(wires = size)
    return qml.expval(qml.PauliZ(wires = size))

def reducedEVA(w):
    return model2(w) * k 

print("expected value:", reducedEVA(w))

expected value: -0.051958823529411766


Finally, we will implement the VQE and its optimized version to check the results through this other approach.


In [5]:
def vqe(w):
    dev2 = qml.device("default.qubit", size, shots = shots)
    cost_fn = qml.ExpvalCost(ansantz, H, dev2, optimize = False)
    return cost_fn(w)

def vqe_compact(w):
    dev2 = qml.device("default.qubit", size , shots = shots)
    cost_fn = qml.ExpvalCost(ansantz, H, dev2, optimize = True)
    return cost_fn(w)

print("VQE, expected value:", vqe(w))
print("VQE optimize, expected value:", vqe_compact(w))

VQE, expected value: -0.05167444310684194
VQE optimize, expected value: -0.05088705702260766


As we can see, the four methods approximate the expected value in a similar way.